In [1]:
import requests, json
from requests.auth import HTTPBasicAuth

In [2]:
# https://docs.google.com/document/d/1txAdY6YChEcdOoEWquF3GzT6yBvpHOiuOJIQvWNZHaE/edit?tab=t.0#heading=h.mij7lb2wckgv
username = "john.f.kennedy@noaa.gov"
password = "############"

url = "https://test-www.fisheries.noaa.gov/inport"

response = requests.get(url, auth=HTTPBasicAuth(username, password))

if response.status_code == 200:
    print("Authentication successful!")
    for line in response.iter_lines(decode_unicode=True):
        if line:
            #print(json.loads(line))
            print(line)
    #for line in response.text.splitlines():
    #    data = json.loads(line)
    #    print(data)
    #print(response.json())
    #print(response.content.decode("utf-8"))
else:
    print(f"Authentication failed. Status code: {response.status_code}")
    print(response.text)


Authentication successful!
<!doctype html>
<html lang="en">
<head>
      <script async src="https://www.googletagmanager.com/gtag/js?id=G-44FQGBN8G1"></script>
  <script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'G-44FQGBN8G1');
  </script>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>InPort | NOAA Fisheries &amp; NOS Metadata Repository | NMFS Enterprise Data Management | InPort</title>
    <meta name="description" content="InPort is the metadata repository of data assets for NOAA Fisheries (NMFS), the National Ocean Service (NOS), and their partners.">
  <link href="//fonts.googleapis.com/css?family=Open+Sans:400,700" rel="stylesheet">
  <link href="//fonts.googleapis.com/css?family=Montserrat:400,600" rel="stylesheet">
  <link rel="icon" type="image/png" href="/inport/images/inport-icon-32.png" />
  <link href="/inport/css